In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

from sklearn.model_selection import KFold
from random import shuffle

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [2]:
#

In [3]:
tf.keras.backend.set_floatx('float64')

In [162]:
model = keras.Sequential(
    [
        keras.Input(shape=(2,)),
        layers.Dense(6, name="layer1", activation='elu'),
        layers.Dense(1, name="layer2"),
    ]
)


In [5]:
# Generate solution

In [6]:
#
N_n = int(2)
c = 0.4
k = 4.0
def oscilator(t, y, f):
    f_val = f(t)
    return np.array([y[1], -c*y[1]-k*y[0]+f_val])
t_final = 200.0
n_eval = int(1000)
t_span = np.array([0.0, t_final])
t_eval = np.linspace(0, t_final, num=n_eval)
y0 = np.array([1.0, 0.0])
#
f_ext = np.vstack(( np.zeros((200,1)), np.ones((400,1))*0.5, np.ones((400,1)) ) )
f_ext += np.random.normal(loc = 0.0, scale = 0.02, size=np.shape(f_ext))
f_interp = interp1d(t_eval, f_ext, kind='linear', axis=0)

In [7]:
func_1 = lambda t,y: oscilator(t,y,f_interp)
sol = solve_ivp(func_1, t_span, y0, t_eval=t_eval)

<ipython-input-6-c26570abacc8>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([y[1], -c*y[1]-k*y[0]+f_val])


In [8]:
plt.plot(sol.t, sol.y[0,:])

In [9]:
# transform to tensorflow
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )
f_ext_tf = tf.constant(f_ext)

In [54]:
# interpolation in tf
f_ext_interp_np = interp1d(t_eval, f_ext, kind='linear', axis=0)
f_ext_interp = lambda t: tf.expand_dims(tf.constant(f_ext_interp_np(t)), axis=0)

In [10]:
# only displacements
y_target = tf.expand_dims(tf.constant(np.transpose(sol.y[0,:])), axis=1)

In [ ]:
# shifted

In [11]:
y_targ_2 = y_target[1:n_eval,:]
y_shift = y_target[0:n_eval-1,:] 
x = tf.concat([y_shift, f_ext_tf[0:n_eval-1,:]], axis=1)

In [164]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.MeanSquaredError())

In [165]:
model.fit(x = x, y = y_targ_2, batch_size = 50, epochs = 200)

Epoch 1/200
20/20 [==============================] - 0s 0s/step - loss: 0.1401
Epoch 2/200
20/20 [==============================] - 0s 0s/step - loss: 0.0643
Epoch 3/200
20/20 [==============================] - 0s 781us/step - loss: 0.0345
Epoch 4/200
20/20 [==============================] - 0s 781us/step - loss: 0.0266
Epoch 5/200
20/20 [==============================] - 0s 781us/step - loss: 0.0236
Epoch 6/200
20/20 [==============================] - 0s 781us/step - loss: 0.0211
Epoch 7/200
20/20 [==============================] - 0s 0s/step - loss: 0.0188
Epoch 8/200
20/20 [==============================] - 0s 781us/step - loss: 0.0168
Epoch 9/200
20/20 [==============================] - 0s 781us/step - loss: 0.0150
Epoch 10/200
20/20 [==============================] - 0s 781us/step - loss: 0.0135
Epoch 11/200
20/20 [==============================] - 0s 781us/step - loss: 0.0121
Epoch 12/200
20/20 [==============================] - 0s 781us/step - loss: 0.0109
Epoch 13/200
20/20 [==

20/20 [==============================] - 0s 781us/step - loss: 8.9794e-04
Epoch 100/200
20/20 [==============================] - 0s 781us/step - loss: 8.9754e-04
Epoch 101/200
20/20 [==============================] - 0s 0s/step - loss: 8.9774e-04
Epoch 102/200
20/20 [==============================] - 0s 1ms/step - loss: 8.9676e-04
Epoch 103/200
20/20 [==============================] - 0s 0s/step - loss: 8.9707e-04
Epoch 104/200
20/20 [==============================] - 0s 781us/step - loss: 8.9424e-04
Epoch 105/200
20/20 [==============================] - 0s 781us/step - loss: 8.9609e-04
Epoch 106/200
20/20 [==============================] - 0s 781us/step - loss: 8.9851e-04
Epoch 107/200
20/20 [==============================] - 0s 781us/step - loss: 8.9734e-04
Epoch 108/200
20/20 [==============================] - 0s 0s/step - loss: 8.9537e-04
Epoch 109/200
20/20 [==============================] - 0s 0s/step - loss: 8.9436e-04
Epoch 110/200
20/20 [==============================] - 0s 1m

20/20 [==============================] - 0s 1ms/step - loss: 8.8477e-04
Epoch 194/200
20/20 [==============================] - 0s 0s/step - loss: 8.8687e-04
Epoch 195/200
20/20 [==============================] - 0s 0s/step - loss: 8.8921e-04
Epoch 196/200
20/20 [==============================] - 0s 781us/step - loss: 8.8544e-04
Epoch 197/200
20/20 [==============================] - 0s 781us/step - loss: 8.9073e-04
Epoch 198/200
20/20 [==============================] - 0s 781us/step - loss: 8.8617e-04
Epoch 199/200
20/20 [==============================] - 0s 781us/step - loss: 8.8849e-04
Epoch 200/200
20/20 [==============================] - 0s 0s/step - loss: 8.8546e-04


In [166]:
y_pred = model.predict(x)

In [172]:
fig = plt.figure()
ax = plt.gca()
ax.plot(y_pred)
ax.plot(y_targ_2.numpy())

In [ ]:
#

In [ ]:
y_pred = [tf.expand_dims(y_shift[0,:], axis=0)]
for i in range(len(y_shift)):
    y_pred.append(model.predict(y_pred[i]))

In [ ]:
# Neural ODE

In [17]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE
import neural_ode.ODESolvers

In [24]:
class SimpleModel(tf.keras.Model):
    def __init__(self, dyn_dim = 1, external_dim=1):
        super().__init__()
        w_init = tf.random_normal_initializer(mean=-1.0, stddev=0.05)
        self.w = tf.Variable(
            initial_value = w_init(shape=(dyn_dim*2, dyn_dim), dtype="float64"),
            trainable=True, name='W' )
        self.dyn_dim = dyn_dim
        w_ext = tf.Variable(
            initial_value = w_init(shape=(external_dim, dyn_dim), dtype="float64"),
            trainable=True, name='W_ext' )
        self.w_ext = w_ext

    def call(self, inputs):
        y_dyn = inputs[:,:self.dyn_dim*2]
        x_ext = inputs[:,self.dyn_dim*2:]
        vels = inputs[:,self.dyn_dim:self.dyn_dim*2]
        accs = tf.matmul(y_dyn, self.w) + tf.matmul(x_ext, self.w_ext)
        return tf.concat([vels, accs], axis=1)


model_2 = SimpleModel()

In [25]:
model_2.variables

[<tf.Variable 'W:0' shape=(2, 1) dtype=float64, numpy=
 array([[-1.04367319],
        [-0.96973507]])>,
 <tf.Variable 'W_ext:0' shape=(1, 1) dtype=float64, numpy=array([[-0.96773713]])>]

In [26]:
model_2(tf.constant([[1.0, 2.0, 2.0]], dtype=tf.float64))

<tf.Tensor: shape=(1, 2), dtype=float64, numpy=array([[ 2.       , -4.9186176]])>

In [27]:
n_ode = neural_ode.NeuralODE.NeuralODE(model_2, 2, n_external=1)

In [35]:
n_epoch = 20
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_batch=10, 
          adjoint_method=False, missing_derivative=[0], adjust_initial=False, x_external=f_ext_tf)

--- Epoch #1 ---
Total loss of epoch: 0.02925563654571306
Elapsed time for epoch: 5.526193857192993
--- Epoch #2 ---
Total loss of epoch: 0.028828491635977116
Elapsed time for epoch: 5.459866762161255
--- Epoch #3 ---
Total loss of epoch: 0.02799747678136555
Elapsed time for epoch: 5.456030368804932
--- Epoch #4 ---
Total loss of epoch: 0.027542313801404816
Elapsed time for epoch: 5.565034866333008
--- Epoch #5 ---
Total loss of epoch: 0.026894787261426245
Elapsed time for epoch: 5.444942474365234
--- Epoch #6 ---
Total loss of epoch: 0.026002113946560712
Elapsed time for epoch: 5.5343017578125
--- Epoch #7 ---
Total loss of epoch: 0.025824300232216046
Elapsed time for epoch: 5.501861333847046
--- Epoch #8 ---
Total loss of epoch: 0.025020309491083026
Elapsed time for epoch: 5.737710952758789
--- Epoch #9 ---
Total loss of epoch: 0.023960097692906857
Elapsed time for epoch: 5.44577956199646
--- Epoch #10 ---
Total loss of epoch: 0.02369606881211439
Elapsed time for epoch: 5.49304771423

In [117]:
n_ode.model.variables

[<tf.Variable 'W:0' shape=(2, 1) dtype=float64, numpy=
 array([[-4. ],
        [-0.4]])>,
 <tf.Variable 'W_ext:0' shape=(1, 1) dtype=float64, numpy=array([[1.]])>]

In [116]:
n_ode.model.variables[0].assign(np.array([[-k],[-c]]))
n_ode.model.variables[1].assign(np.array([[1.0]]))

<tf.Variable 'UnreadVariable' shape=(1, 1) dtype=float64, numpy=array([[1.]])>

In [169]:
y0 = tf.concat([y_target[0,0], (y_target[1,0]-y_target[0,0])/(t_target[1]-t_target[0])], axis=0)
sol_model = n_ode.forward_solve(t_target, y0, x_external=f_ext_interp)

In [171]:
fig, ax = plt.subplots()
plt.plot(t_target.numpy(), y_target.numpy())
ax.plot(sol_model['t'].numpy(), sol_model['y'][:,0].numpy() )

# Test

In [205]:
#
N_n = int(2)
c1 = 0.2
k1 = 4.0
def oscilator(t, y, f):
    f_val = f(t)
    return np.array([y[1], -c1*y[1]-k1*y[0]+f_val])
t_final = 200.0
n_eval = int(1000)
t_span = np.array([0.0, t_final])
t_eval = np.linspace(0, t_final, num=n_eval)
y0 = np.array([1.0, 0.0])

In [206]:
func_1 = lambda t,y: oscilator(t,y,f_interp)
sol_test = solve_ivp(func_1, t_span, y0, t_eval=t_eval)

<ipython-input-205-18b09cb65a89>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([y[1], -c1*y[1]-k1*y[0]+f_val])


In [207]:
plt.plot(sol_test.t, sol_test.y[0,:])

In [208]:
# transform to tensorflow
t_test = tf.constant(sol_test.t)
y_test = tf.expand_dims(tf.constant(np.transpose(sol_test.y[0,:])), axis=1)

In [209]:
# usual model

In [210]:
# shift
y_test_out = y_test[1:n_eval,:]
y_test_inp = y_test[0:n_eval-1,:] 
x_test = tf.concat([y_test_inp, f_ext_tf[0:n_eval-1,:]], axis=1)

In [211]:
y_pred_test = model.predict(x_test)

In [212]:
# neural ODE model

In [213]:
y0 = tf.concat([y_test[0,0], (y_test[1,0]-y_test[0,0])/(t_target[1]-t_target[0])], axis=0)
sol_model_test = n_ode.forward_solve(t_target, y0, x_external=f_ext_interp)

In [214]:
y_pred_test_2 = sol_model_test['y'][::n_ode.n_ref,0].numpy()

In [215]:
y_test_np = np.reshape(y_test.numpy(), (len(y_test.numpy())) )

In [221]:
def resudual_func(y_targ, y_pr):
    dict_res = {}
    dict_res['residual'] = np.abs(y_targ - y_pr)
    dict_res['residual_rel'] =  dict_res['residual']/(np.abs(y_targ)+np.abs(y_pr)+ 1e-5 )*2
    return dict_res

In [222]:
res1 = resudual_func(y_test_out, y_pred_test)

In [223]:
res2 = resudual_func(y_test_np, y_pred_test_2)

In [226]:
fig, ax = plt.subplots()
ax.plot(y_test_out.numpy())
ax.plot(y_pred_test)
ax.plot(res1['residual_rel'])

In [227]:
fig, ax = plt.subplots()
ax.plot(y_test_np)
ax.plot(y_pred_test_2)
ax.plot(res2['residual_rel'])